In [1]:
import cobra
import sys
import pandas as pd
import os
from itertools import chain
# currently working with local medusa installation -- anytime changes are made to medusa, we need to run installation
# again within the virtualenv by running setup.py
#import medusa
from medusa.reconstruct.expand.expand import iterative_gapfill_from_binary_phenotypes
from cobra.flux_analysis import flux_variability_analysis
from cobra.flux_analysis.parsimonious import add_pfba

import itertools
import random
from itertools import chain

from optlang.interface import OPTIMAL

from cobra.flux_analysis.gapfilling import GapFiller
from cobra.flux_analysis.parsimonious import add_pfba
from cobra.core import DictList

from cobra.util.solver import linear_reaction_coefficients

from medusa.core.ensemble import Ensemble
from medusa.core.feature import Feature
from medusa.core.member import Member


/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/greg/Envs/ssl_ensembles/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
biolog_base_composition = pd.read_csv('../data/biolog_base_composition.csv',sep=',')
biolog_base_dict = dict(zip(biolog_base_composition['ID'],\
                          [1000 for i in range(0,len(biolog_base_composition['ID']))]))
biolog_thresholded = pd.read_csv('../data/plata_thresholded.csv',sep='\t',index_col=0)

In [3]:
universal = cobra.io.load_json_model('../data/universal_mundy.json')

In [4]:
testmod = cobra.io.load_json_model('../data/modelseed_models/Staphylococcus aureus.json')

In [5]:
testmod.optimize()
testmod_removed = testmod.copy()
testmod_removed.remove_reactions(testmod_removed.reactions[0:20])
testmod_removed.optimize()

,fluxes,reduced_costs
rxn05667_c,0.0,-0.000000e+00
rxn00322_c,0.0,0.000000e+00
rxn05457_c,0.0,-0.000000e+00
rxn04271_c,0.0,-0.000000e+00
rxn13783_c,0.0,0.000000e+00
...,...,...
EX_cpd00066_e,0.0,0.000000e+00
EX_cpd00099_e,0.0,0.000000e+00
EX_cpd01080_e,0.0,0.000000e+00
SK_cpd11416_c,0.0,0.000000e+00


In [6]:
add_pfba(universal)

In [7]:
rxns_to_remove = [rxn for rxn in universal.reactions if rxn.id in [rxn.id for rxn in testmod_removed.reactions]]
universal.remove_reactions(rxns_to_remove)
universal.add_reactions([rxn for rxn in testmod_removed.reactions])

In [8]:
universal.reactions.get_by_id('bio1').lower_bound = 0.05

In [9]:
solution = universal.optimize()

In [10]:
get_fluxes = set([r.id for r in universal.reactions]) - set([r.id for r in rxns_to_remove])
{rxn:solution.x_dict[rxn] for rxn in get_fluxes if abs(solution.x_dict[rxn]) > 1E-8}

{'EX_cpd00007_e': -0.9058290334221217,
 'EX_cpd00010_e': -0.0007939658594680477,
 'EX_cpd00023_e': 1.1647557981410817,
 'EX_cpd00028_e': -0.0007939658594680431,
 'EX_cpd00030_e': -0.00039698292973402153,
 'EX_cpd00033_e': 0.1172885528955931,
 'EX_cpd00034_e': -0.00039698292973402153,
 'EX_cpd00039_e': -0.016275074099941857,
 'EX_cpd00041_e': -0.2297000914703219,
 'EX_cpd00048_e': -0.0003969829297340219,
 'EX_cpd00051_e': -0.06253894645571907,
 'EX_cpd00058_e': -0.00039698292973402153,
 'EX_cpd00060_e': -0.009316579548261811,
 'EX_cpd00063_e': -0.00039698292973402153,
 'EX_cpd00064_e': 0.06253894645571907,
 'EX_cpd00066_e': -0.0088094324187051,
 'EX_cpd00067_e': -1.1887279165364748,
 'EX_cpd00069_e': -0.00555199112899785,
 'EX_cpd00080_e': -0.05893200134095822,
 'EX_cpd00092_e': 0.40311473592086616,
 'EX_cpd00099_e': -0.00039698292973402153,
 'EX_cpd00119_e': -0.004097410427304695,
 'EX_cpd00129_e': -1.2102558372921297,
 'EX_cpd00149_e': -0.00039698292973402153,
 'EX_cpd00159_e': 0.2064

In [11]:
# verify that the model produces biomass after we add the suggested reactions
testmod_removed.optimize()

,fluxes,reduced_costs
rxn05667_c,0.0,-0.000000e+00
rxn00322_c,0.0,0.000000e+00
rxn05457_c,0.0,-0.000000e+00
rxn04271_c,0.0,-0.000000e+00
rxn13783_c,0.0,0.000000e+00
...,...,...
EX_cpd00066_e,0.0,0.000000e+00
EX_cpd00099_e,0.0,0.000000e+00
EX_cpd01080_e,0.0,0.000000e+00
SK_cpd11416_c,0.0,0.000000e+00


In [12]:
testmod_removed_restored = testmod_removed.copy()
testmod_removed_restored.add_reactions([universal.reactions.get_by_id('rxn03408_c').copy(),\
                                       universal.reactions.get_by_id('rxn05440_c').copy(),\
                                       universal.reactions.get_by_id('rxn23650_c').copy()])
testmod_removed_restored.optimize()

,fluxes,reduced_costs
rxn05667_c,-1000.000000,-6.215068e-03
rxn00322_c,0.000000,-2.168404e-19
rxn05457_c,0.000000,0.000000e+00
rxn04271_c,0.000000,-8.611211e-19
rxn13783_c,93.459087,0.000000e+00
...,...,...
SK_cpd11416_c,93.459087,0.000000e+00
bio1,93.459087,-2.134923e-15
rxn03408_c,0.168793,0.000000e+00
rxn05440_c,7.770704,-8.673617e-18


In [13]:
# how can we update the linear coefficients associated with a set of variables, rather than resetting for all?
#

In [14]:
coefficients = universal.objective.get_linear_coefficients(universal.variables)
#for reaction in universal.reactions:

reaction_variables = ((universal.reactions.get_by_id(rxn).forward_variable,
                                        universal.reactions.get_by_id(rxn).reverse_variable)
                                        for rxn in ['rxn03408_c','rxn05440_c','rxn23650_c'])
variables = chain(*reaction_variables)
for variable in variables:
    coefficients[variable] = 0.0

universal.objective.set_linear_coefficients(coefficients)
universal.optimize()

,fluxes,reduced_costs
rxn31485_c,0.000000,-2.000000e+00
rxn00594_c,0.000000,-1.250000e-01
rxn25971_c,0.000000,-2.000000e+00
rxn01087_c,0.000000,2.000000e+00
rxn38695_c,0.000000,0.000000e+00
...,...,...
EX_cpd00066_e,-0.008809,0.000000e+00
EX_cpd00099_e,-0.000397,0.000000e+00
EX_cpd01080_e,-0.004157,0.000000e+00
SK_cpd11416_c,0.050000,0.000000e+00


In [15]:
# load all models, perform FVA, and determine which metabolites are essential in the majority of all models
species_to_essential = {}
for modelfile in os.listdir('../data/modelseed_models/'):
    species_name = modelfile.split('/')[-1].split('.')[0]
    print("performing FVA for " + species_name)
    model = cobra.io.load_json_model('../data/modelseed_models/'+modelfile)
    exchanges = model.exchanges
    for rxn in exchanges:
        rxn.lower_bound = -1000
    fva_result = flux_variability_analysis(model=model,fraction_of_optimum=0.01,reaction_list=exchanges)
    essentials = list(fva_result.loc[fva_result['maximum'] < -0.0001].index)
    species_to_essential[species_name] = essentials

performing FVA for Neisseria gonorrhoeae
performing FVA for Bacillus megaterium
performing FVA for Streptococcus mitis
performing FVA for Corynebacterium glutamicum
performing FVA for Enterococcus faecalis
performing FVA for Streptococcus equinus
performing FVA for Bacillus pumilus
performing FVA for Corynebacterium efficiens
performing FVA for Flavobacterium johnsoniae
performing FVA for Stenotrophomonas maltophilia
performing FVA for Haemophilus influenzae
performing FVA for Pseudomonas stutzeri
performing FVA for Neisseria flavescens
performing FVA for Staphylococcus aureus
performing FVA for Streptococcus oralis
performing FVA for Haemophilus parasuis
performing FVA for Staphylococcus epidermidis
performing FVA for Salmonella enterica subsp
performing FVA for Streptococcus gallolyticus
performing FVA for Chryseobacterium gleum
performing FVA for Bacillus subtilis subsp
performing FVA for Gordonia bronchialis
performing FVA for Achromobacter xylosoxidans
performing FVA for Listeria 

In [16]:
count_essential = {}
for species in species_to_essential.keys():
    essentials = species_to_essential[species]
    for metabolite in essentials:
        if metabolite in count_essential.keys():
            count_essential[metabolite] +=1
        else:
            count_essential[metabolite] = 1

In [17]:
majority_essential = [exchange for exchange in count_essential.keys() if \
                      count_essential[exchange] == len(species_to_essential.keys())]
majority_essential_as_compounds = [x[3:] for x in majority_essential]
for met in majority_essential_as_compounds:
    print(universal.metabolites.get_by_id(met).name)
    if met not in biolog_base_dict.keys():
        print(met)
        #biolog_base_dict[met] = 1000

Zn2+
Co2+
Heme
fe3
Cl-
Ca2+
Mn2+
ocdca
cpd01080_e
K+
Cu2+
Mg


In [18]:
biolog_base_dict

{'cpd00001_e': 1000,
 'cpd00007_e': 1000,
 'cpd00009_e': 1000,
 'cpd00011_e': 1000,
 'cpd00013_e': 1000,
 'cpd00028_e': 1000,
 'cpd00030_e': 1000,
 'cpd00034_e': 1000,
 'cpd00048_e': 1000,
 'cpd00058_e': 1000,
 'cpd00063_e': 1000,
 'cpd00067_e': 1000,
 'cpd00099_e': 1000,
 'cpd00149_e': 1000,
 'cpd00205_e': 1000,
 'cpd00254_e': 1000,
 'cpd00268_e': 1000,
 'cpd00971_e': 1000,
 'cpd10515_e': 1000,
 'cpd10516_e': 1000}

In [19]:
test_mod_pheno = biolog_thresholded.loc['Staphylococcus aureus']
test_mod_pheno = list(test_mod_pheno[test_mod_pheno == True].index)

In [20]:
# check for biolog base components in the model
add_mets = []
add_exchanges = []
for met in list(biolog_base_dict.keys()):
    try:
        testmod.metabolites.get_by_id(met)
    except:
        print('no '+met)
        add_met = universal.metabolites.get_by_id(met).copy()
        add_mets.append(add_met)

testmod.add_metabolites(add_mets)

for met in list(biolog_base_dict.keys()):
    # Search for exchange reactions
    try:
        testmod.reactions.get_by_id('EX_'+met)
    except:
        add_met = testmod.metabolites.get_by_id(met)
        ex_rxn = cobra.core.Reaction('EX_' + met)
        ex_rxn.name = "Exchange reaction for " + met
        ex_rxn.lower_bound = -1000
        ex_rxn.upper_bound = 1000
        ex_rxn.add_metabolites({add_met:-1})
        add_exchanges.append(ex_rxn)

    

testmod.add_reactions(add_exchanges)

no cpd00013_e


In [21]:
# Grab 5 carbon sources and generate media dictionaries for them
missing_mets = []
missing_exchanges = []
media_dicts = {}
for met_id in test_mod_pheno:
    try:
        testmod.metabolites.get_by_id(met_id)
    except:
        print(met_id + " was not in model, adding met and exchange reaction")
        met = universal.metabolites.get_by_id(met_id).copy()
        missing_mets.append(met)
        ex_rxn = cobra.core.Reaction('EX_' + met_id)
        ex_rxn.name = "Exchange reaction for " + met_id
        ex_rxn.lower_bound = -1000
        ex_rxn.upper_bound = 1000
        ex_rxn.add_metabolites({met:-1})
        missing_exchanges.append(ex_rxn)
    media_dicts[met_id] = biolog_base_dict.copy()
    media_dicts[met_id] = {'EX_'+k:v for k,v in media_dicts[met_id].items()}
    media_dicts[met_id]['EX_'+met_id] = 1000
testmod.add_metabolites(missing_mets)
testmod.add_reactions(missing_exchanges)


cpd11594_e was not in model, adding met and exchange reaction
cpd05158_e was not in model, adding met and exchange reaction
cpd15584_e was not in model, adding met and exchange reaction
cpd00492_e was not in model, adding met and exchange reaction
cpd00232_e was not in model, adding met and exchange reaction
cpd19001_e was not in model, adding met and exchange reaction
cpd00709_e was not in model, adding met and exchange reaction
cpd00396_e was not in model, adding met and exchange reaction
cpd01307_e was not in model, adding met and exchange reaction
cpd00079_e was not in model, adding met and exchange reaction
cpd00072_e was not in model, adding met and exchange reaction
cpd00320_e was not in model, adding met and exchange reaction
cpd01293_e was not in model, adding met and exchange reaction
cpd05264_e was not in model, adding met and exchange reaction
cpd03561_e was not in model, adding met and exchange reaction
cpd00094_e was not in model, adding met and exchange reaction
cpd00142

In [22]:
transporters_in_universal = [rxn for rxn in universal.reactions if len(rxn.compartments)>1]
for met in media_dicts.keys():
    metabolite = testmod.metabolites.get_by_id(met)
    base_met_id = met.split('_')[0]
    rxns_with_metabolite = metabolite.reactions
    transport = False
    for rxn in rxns_with_metabolite:
        metabolites = [met_in_rxn.id for met_in_rxn in rxn.metabolites]
        if (base_met_id+'_e' in metabolites and base_met_id+'_c' in metabolites):
            transport = True
    
    pick_transporter = {}
    if not transport:
        print("missing transporter for " + metabolite.name)
        for rxn in transporters_in_universal:
            metabolites = [met_in_rxn.id for met_in_rxn in rxn.metabolites]
            if (base_met_id+'_e' in metabolites and base_met_id+'_c' in metabolites):
                pick_transporter[met] = rxn.id
                print("found transporter for "+metabolite.name)
                print("transporter is " + str(rxn.reversibility))

missing transporter for D-Mannose
found transporter for D-Mannose
transporter is True
found transporter for D-Mannose
transporter is True
found transporter for D-Mannose
transporter is True
found transporter for D-Mannose
transporter is True
missing transporter for 5-Oxoproline
found transporter for 5-Oxoproline
transporter is True
missing transporter for D-Fructose
found transporter for D-Fructose
transporter is True
found transporter for D-Fructose
transporter is True
missing transporter for Neu5Ac
found transporter for Neu5Ac
transporter is True
missing transporter for D-Lyxitol
found transporter for D-Lyxitol
transporter is True
found transporter for D-Lyxitol
transporter is True
found transporter for D-Lyxitol
transporter is False
missing transporter for 2-Oxobutyrate
found transporter for 2-Oxobutyrate
transporter is True
missing transporter for N-Acetyl-D-glucosamine
found transporter for N-Acetyl-D-glucosamine
transporter is True
missing transporter for Acetate
found transporte

In [23]:
# reload the universal to start without the pfba constraints/objective
universal = cobra.io.load_json_model('../data/universal_mundy.json')

In [24]:
# Add the transporters to the model
transporters_to_add = list(pick_transporter.values())
transporter_list = []
for rxn in transporters_to_add:
    transporter_list.append(universal.reactions.get_by_id(rxn).copy())
testmod.add_reactions(transporter_list)

# remove the added transporters from the universal model
universal.remove_reactions([universal.reactions.get_by_id(rxn) for rxn in transporters_to_add])

In [25]:
optcomplete = testmod.optimize().f

In [26]:
for rxn in testmod.exchanges:
    rxn.lower_bound = -1000
optcomplete = testmod.optimize().f

In [27]:
fva_result = flux_variability_analysis(model=testmod,fraction_of_optimum=0.05,reaction_list=testmod.exchanges)

In [28]:
essentials = list(fva_result.loc[fva_result['maximum'] < -0.00001].index)
for rxn in essentials:
    mets = testmod.reactions.get_by_id(rxn).metabolites
    for met in mets:
        print(met.id,met.name)
#[[met.name for met in mets] for mets in rxn.metabolites for rxn in essentials]

cpd00058_e Cu2+
cpd10515_e Fe2+
cpd00179_e Maltose
cpd00060_e L-Methionine
cpd00034_e Zn2+
cpd10516_e fe3
cpd03696_e Ursin
cpd00277_e Deoxyguanosine
cpd00220_e Riboflavin
cpd00028_e Heme
cpd00254_e Mg
cpd00205_e K+
cpd00039_e L-Lysine
cpd00030_e Mn2+
cpd00149_e Co2+
cpd00305_e Thiamin
cpd00264_e Spermidine
cpd00069_e L-Tyrosine
cpd00268_e H2S2O3
cpd00063_e Ca2+
cpd00393_e Folate
cpd00355_e Nicotinamide ribonucleotide
cpd00048_e Sulfate
cpd00119_e L-Histidine
cpd00066_e L-Phenylalanine
cpd00099_e Cl-
cpd01080_e ocdca


In [29]:
sources = list(media_dicts.keys())
sub_dict = {sources[0]:media_dicts[sources[0]],
           sources[1]:media_dicts[sources[1]],
           sources[2]:media_dicts[sources[2]],
           sources[3]:media_dicts[sources[3]],
           sources[4]:media_dicts[sources[4]]}

In [30]:
media_dicts[sources[3]]

{'EX_cpd00001_e': 1000,
 'EX_cpd00007_e': 1000,
 'EX_cpd00009_e': 1000,
 'EX_cpd00011_e': 1000,
 'EX_cpd00013_e': 1000,
 'EX_cpd00028_e': 1000,
 'EX_cpd00030_e': 1000,
 'EX_cpd00034_e': 1000,
 'EX_cpd00041_e': 1000,
 'EX_cpd00048_e': 1000,
 'EX_cpd00058_e': 1000,
 'EX_cpd00063_e': 1000,
 'EX_cpd00067_e': 1000,
 'EX_cpd00099_e': 1000,
 'EX_cpd00149_e': 1000,
 'EX_cpd00205_e': 1000,
 'EX_cpd00254_e': 1000,
 'EX_cpd00268_e': 1000,
 'EX_cpd00971_e': 1000,
 'EX_cpd10515_e': 1000,
 'EX_cpd10516_e': 1000}

In [31]:
#





In [32]:
seed_rxn_table = pd.read_csv('../data/reactions_seed_20180809.tsv',sep='\t')
seed_rxn_table['id'] = seed_rxn_table['id'] + '_c'

In [33]:
ok_ids = list(seed_rxn_table.loc[(seed_rxn_table['status'] == 'OK') | (seed_rxn_table['status'] == 'HB')]['id'])
remove_rxns = []
for reaction in universal.reactions:
    if reaction.id not in ok_ids:
        remove_rxns.append(reaction)
len(remove_rxns)

5716

In [34]:
universal.remove_reactions(remove_rxns)
mets_in_reactions = []
for reaction in universal.reactions:
    mets = [met.id for met in reaction.metabolites]
    mets_in_reactions.extend(mets)
mets_in_reactions = set(mets_in_reactions)
len(mets_in_reactions)

11373

In [35]:
mets_missing_reactions = []
for metabolite in universal.metabolites:
    if metabolite.id not in mets_in_reactions:
        mets_missing_reactions.append(metabolite)
universal.remove_metabolites(mets_missing_reactions)

In [36]:
# remove all reactions associated wiht metabolites that have mass balance issues across modelseed
universal_mets = [met.id for met in universal.metabolites]
remove_rxns = []
remove_mets = []
for met in ['cpd11657']:
    if met+'_c' in universal_mets:
        rxns = [rxn for rxn in universal.metabolites.get_by_id(met+'_c').reactions]
        remove_rxns.extend(rxns)
        remove_mets.append(universal.metabolites.get_by_id(met+'_c'))
    if met+'_e' in universal_mets:
        rxns = [rxn for rxn in universal.metabolites.get_by_id(met+'_e').reactions]
        remove_rxns.extend(rxns)
        remove_mets.append(universal.metabolites.get_by_id(met+'_e'))

universal.remove_reactions(remove_rxns)
universal.remove_metabolites(remove_mets)

cobra/core/model.py:613 UserWarning: rxn31357_c: cpd11657_c <=> cpd11657_e not in MicrobialNegative


In [37]:
# # add exchange reactions allowing flux out of the system for all extracellular metabolite in the 
# # universal reaction bag
# print(len(universal.reactions))
# for metabolite in universal.metabolites:
#     if metabolite.compartment == 'e':
#         rxn = universal.add_boundary(
#         metabolite, type='exchange_gapfilling', lb=0,ub=1000,
#         reaction_id='EX_{}'.format(metabolite.id))
#         # create exchange reaction for metabolite
# print(len(universal.reactions))

In [38]:
# # can we find a solution even with demand reactions enabled?
# # create a demand reaction for every metabolite
# for metabolite in universal.metabolites:
#     if metabolite.compartment == 'c':
#         rxn = universal.add_boundary(
#         metabolite, type='demand_gapfilling', lb=0,ub=1000,
#         reaction_id='DM_{}'.format(metabolite.id))
# print(len(universal.reactions))

In [39]:

universal.repair()

In [40]:
ensemble = iterative_gapfill_from_binary_phenotypes(testmod,universal,media_dicts,5,\
                                         lower_bound=0.05,\
                                         inclusion_threshold=1E-8,\
                                         exchange_reactions=False,\
                                         demand_reactions=False)

Constraining lower bound for bio1
starting cycle number 0
<Solution 0.994 at 0x7f450683a5c0>
{'rxn03909_c': 0.0003969829297340203, 'rxn10960_c': -0.5016843415351157, 'rxn00958_c': 0.00039698292973402164, 'rxn01547_c': -0.00044214533007901156, 'rxn12239_c': 0.0003969829297340203, 'rxn01256_c': 0.014758406477436968, 'rxn20662_c': 0.0007939658594682048, 'rxn05229_c': 0.009550586485283523, 'rxn01072_c': -0.0003969829297342382, 'rxn08336_c': 0.0040613514564050626, 'rxn00605_c': -0.015441797967100825, 'rxn01014_c': -0.0007939658594678227, 'rxn02834_c': 0.004097410427304695, 'rxn00916_c': 0.004105046607631349, 'rxn10030_c': 0.004157275598057512, 'rxn13141_c': -0.00993332032971397, 'rxn01644_c': 0.016275074099941857, 'rxn05466_c': 0.37170483150215033, 'rxn12215_c': 0.00845379394995515, 'rxn05039_c': 0.0007939658594680431, 'rxn02835_c': 0.004097410427304695, 'rxn08131_c': 0.0003969829297340203, 'rxn02928_c': -0.016275074099941857, 'rxn02160_c': 0.004097410427304695, 'rxn02250_c': -0.00039698292

reactions included after gapfilling on cpd00208_e:
{'rxn10960_c', 'rxn00958_c', 'rxn12239_c', 'rxn20662_c', 'rxn05229_c', 'rxn08336_c', 'rxn00605_c', 'rxn00916_c', 'rxn13141_c', 'rxn01644_c', 'rxn05466_c', 'rxn12215_c', 'rxn08131_c', 'rxn01014_c', 'rxn02250_c', 'rxn03909_c', 'rxn01547_c', 'rxn01256_c', 'rxn01072_c', 'rxn02834_c', 'rxn05039_c', 'rxn09989_c', 'rxn10030_c', 'rxn02835_c', 'rxn02928_c', 'rxn02160_c', 'rxn05531_c'}
<Solution 0.000 at 0x7f4506843a20>
{'rxn03909_c': 0.00039698292973402034, 'rxn00958_c': 0.00039698292973402055, 'rxn12239_c': 0.0003969829297340203, 'rxn01256_c': 0.014758406477436968, 'rxn20662_c': 0.0007939658594680037, 'rxn05229_c': 0.009550586485282574, 'rxn01072_c': -0.00039698292973398066, 'rxn08336_c': 0.00406135145640504, 'rxn00605_c': -0.015441797967100835, 'rxn01014_c': -0.0007939658594680845, 'rxn02834_c': 0.004097410427304695, 'rxn00916_c': 0.004547191937710364, 'rxn10030_c': 0.004157275598057513, 'rxn13141_c': -0.009933320329713991, 'rxn01644_c': 0.01

reactions included after gapfilling on cpd00094_e:
{'rxn10960_c', 'rxn00958_c', 'rxn12239_c', 'rxn05487_c', 'rxn20662_c', 'rxn05229_c', 'rxn08336_c', 'rxn00605_c', 'rxn00916_c', 'rxn13141_c', 'rxn01644_c', 'rxn05466_c', 'rxn12215_c', 'rxn08131_c', 'rxn01014_c', 'rxn02250_c', 'rxn21111_c', 'rxn03909_c', 'rxn01547_c', 'rxn01256_c', 'rxn01072_c', 'rxn02834_c', 'rxn05039_c', 'rxn09989_c', 'rxn10030_c', 'rxn02835_c', 'rxn29410_c', 'rxn02928_c', 'rxn02160_c', 'rxn05531_c'}
<Solution 0.722 at 0x7f45067c1278>
{'rxn03909_c': 0.00039698292973402034, 'rxn00958_c': 0.0003969829297340203, 'rxn01547_c': 0.05644868601806392, 'rxn12239_c': 0.0003969829297340203, 'rxn01256_c': 0.014758406477436968, 'rxn20662_c': 0.0007939658594682961, 'rxn04932_c': 0.02580832479272061, 'rxn29748_c': 0.35200408111051307, 'rxn01072_c': -0.00039698292973429567, 'rxn08336_c': 0.004061351456405039, 'rxn00605_c': -0.01544179796710084, 'rxn01014_c': -0.0007939658594677806, 'rxn02834_c': 0.004097410427304695, 'rxn00916_c': 0.0

<Solution -0.000 at 0x7f45067ddcc0>
{'rxn03909_c': 0.00039698292973402034, 'rxn00958_c': 0.0003969829297340202, 'rxn12239_c': 0.00039698292973402023, 'rxn05487_c': -0.0075717744874318335, 'rxn01256_c': 0.014758406477436968, 'rxn20662_c': 0.0007939658594680698, 'rxn05229_c': 0.009550586485283974, 'rxn01072_c': -0.00039698292973404045, 'rxn08336_c': 0.004061351456405049, 'rxn00605_c': -0.01544179796710084, 'rxn01014_c': -0.0007939658594680145, 'rxn02834_c': 0.004097410427304695, 'rxn00916_c': 0.004547191937710318, 'rxn10030_c': 0.004157275598057513, 'rxn13141_c': -0.017505094817145822, 'rxn01644_c': 0.016275074099941857, 'rxn08131_c': 0.00039698292973402066, 'rxn12215_c': 0.00845379394995515, 'rxn05039_c': 0.000793965859468043, 'rxn02835_c': 0.004097410427304695, 'rxn02928_c': -0.016275074099941857, 'rxn02160_c': 0.004097410427304695, 'rxn02250_c': -0.0003969829297340216}
[<Reaction rxn03909_c at 0x7f4506843da0>, <Reaction rxn00958_c at 0x7f4506752f60>, <Reaction rxn12239_c at 0x7f450675

reactions included after gapfilling on cpd00079_e:
{'rxn10960_c', 'rxn00958_c', 'rxn29748_c', 'rxn12239_c', 'rxn05487_c', 'rxn20662_c', 'rxn05229_c', 'rxn04932_c', 'rxn04942_c', 'rxn08336_c', 'rxn00605_c', 'rxn00916_c', 'rxn13141_c', 'rxn01644_c', 'rxn05466_c', 'rxn12215_c', 'rxn08131_c', 'rxn01014_c', 'rxn02250_c', 'rxn21111_c', 'rxn03909_c', 'rxn01547_c', 'rxn01256_c', 'rxn01072_c', 'rxn02834_c', 'rxn05039_c', 'rxn09989_c', 'rxn10030_c', 'rxn02835_c', 'rxn29410_c', 'rxn02928_c', 'rxn02160_c', 'rxn12522_c', 'rxn05531_c'}
<Solution 0.334 at 0x7f4506843550>
{'rxn03909_c': 0.00039698292973402603, 'rxn00958_c': 0.00039698292973402587, 'rxn12575_c': 0.3344562276900763, 'rxn12239_c': 0.000396982929734026, 'rxn01256_c': 0.014758406477436974, 'rxn20662_c': 0.000793965859468051, 'rxn04932_c': -0.15684024279424907, 'rxn04942_c': 0.15684024279424907, 'rxn01072_c': -0.0003969829297340294, 'rxn08336_c': 0.004061351456405044, 'rxn00605_c': -0.01544179796710084, 'rxn01014_c': -0.0007939658594680355,

...

reactions included after gapfilling on cpd00159_e:
{'rxn00958_c', 'rxn12239_c', 'rxn38867_c', 'rxn38903_c', 'rxn20662_c', 'rxn12303_c', 'rxn01644_c', 'rxn08131_c', 'rxn05466_c', 'rxn12215_c', 'rxn13648_c', 'rxn01014_c', 'rxn02250_c', 'rxn03909_c', 'rxn12530_c', 'rxn01256_c', 'rxn01359_c', 'rxn01072_c', 'rxn38868_c', 'rxn00555_c', 'rxn02834_c', 'rxn05039_c', 'rxn10030_c', 'rxn02835_c', 'rxn10053_c', 'rxn00348_c', 'rxn20627_c', 'rxn02928_c', 'rxn02160_c'}
<Solution -0.000 at 0x7f4506858f28>
{'rxn03909_c': 0.00039698292973402034, 'rxn00958_c': 0.00039698292973402023, 'rxn38867_c': -0.0004421453300790115, 'rxn12239_c': 0.0003969829297340203, 'rxn12215_c': 0.008453793949955149, 'rxn01256_c': 0.014758406477436968, 'rxn20662_c': 0.000793965859467403, 'rxn12303_c': -0.015441797967100792, 'rxn01072_c': -0.0004293565376688788, 'rxn01014_c': -0.0007615922515331896, 'rxn38868_c': 0.004105046607631357, 'rxn01359_c': 0.0040613514564050495, 'rxn00555_c': 0.00993332032971399, 'rxn05039_c': 0.000793965

reactions included after gapfilling on cpd00492_e:
{'rxn00958_c', 'rxn12239_c', 'rxn38867_c', 'rxn38903_c', 'rxn20662_c', 'rxn12303_c', 'rxn01644_c', 'rxn08131_c', 'rxn05466_c', 'rxn12215_c', 'rxn13648_c', 'rxn01014_c', 'rxn02250_c', 'rxn03909_c', 'rxn12530_c', 'rxn01256_c', 'rxn01359_c', 'rxn01072_c', 'rxn38868_c', 'rxn00555_c', 'rxn02834_c', 'rxn05039_c', 'rxn10030_c', 'rxn02835_c', 'rxn10053_c', 'rxn00348_c', 'rxn20627_c', 'rxn02928_c', 'rxn02160_c'}
<Solution -0.000 at 0x7f45068583c8>
{'rxn03909_c': 0.00039698292973402034, 'rxn00958_c': 0.00039698292973402023, 'rxn38867_c': -0.0004421453300790115, 'rxn12239_c': 0.0003969829297340203, 'rxn12215_c': 0.008453793949955149, 'rxn01256_c': 0.014758406477436968, 'rxn20662_c': 0.000793965859467403, 'rxn12303_c': -0.015441797967100792, 'rxn01072_c': -0.0004293565376688788, 'rxn01014_c': -0.0007615922515331896, 'rxn38868_c': 0.004105046607631357, 'rxn01359_c': 0.0040613514564050495, 'rxn00555_c': 0.00993332032971399, 'rxn05039_c': 0.000793965

reactions included after gapfilling on cpd00094_e:
{'rxn00958_c', 'rxn12239_c', 'rxn38867_c', 'rxn38903_c', 'rxn20662_c', 'rxn12303_c', 'rxn01644_c', 'rxn08131_c', 'rxn05466_c', 'rxn12215_c', 'rxn13648_c', 'rxn01014_c', 'rxn02250_c', 'rxn03909_c', 'rxn12530_c', 'rxn01256_c', 'rxn01359_c', 'rxn01072_c', 'rxn38868_c', 'rxn00555_c', 'rxn02834_c', 'rxn05039_c', 'rxn10030_c', 'rxn02835_c', 'rxn10053_c', 'rxn00348_c', 'rxn20627_c', 'rxn02928_c', 'rxn02160_c'}
<Solution -0.000 at 0x7f45067c7668>
{'rxn03909_c': 0.00039698292973402034, 'rxn00958_c': 0.00039698292973402023, 'rxn38867_c': -0.0004421453300790115, 'rxn12239_c': 0.0003969829297340203, 'rxn12215_c': 0.008453793949955149, 'rxn01256_c': 0.014758406477436968, 'rxn20662_c': 0.000793965859467403, 'rxn12303_c': -0.015441797967100792, 'rxn01072_c': -0.0004293565376688788, 'rxn01014_c': -0.0007615922515331896, 'rxn38868_c': 0.004105046607631357, 'rxn01359_c': 0.0040613514564050495, 'rxn00555_c': 0.00993332032971399, 'rxn05039_c': 0.000793965

reactions included after gapfilling on cpd00314_e:
{'rxn00958_c', 'rxn12239_c', 'rxn38867_c', 'rxn38903_c', 'rxn20662_c', 'rxn12303_c', 'rxn01644_c', 'rxn08131_c', 'rxn05466_c', 'rxn12215_c', 'rxn13648_c', 'rxn01014_c', 'rxn02250_c', 'rxn03909_c', 'rxn12530_c', 'rxn01256_c', 'rxn01359_c', 'rxn01072_c', 'rxn38868_c', 'rxn00555_c', 'rxn02834_c', 'rxn05039_c', 'rxn10030_c', 'rxn02835_c', 'rxn10053_c', 'rxn00348_c', 'rxn20627_c', 'rxn02928_c', 'rxn02160_c'}


AttributeError: 'str' object has no attribute 'id'